### Install Sqitch Docker image

Install Sqitch:
```bash
sudo docker pull sqitch/sqitch
sudo curl -L https://git.io/fAX6Z -o sqitch && chmod +x sqitch
sudo cp sqitch /usr/local/bin
```

Allow current user to run docker:

```bash
sudo groupadd docker
sudo gpasswd -a $USER docker
sqitch --version
```

### Install Sqitch system-wide using cpan

This is used since docker install gives problems with permissions (runs under "sqitch" user in docker).
```bash
sudo apt-get install build-essential cpanminus perl perl-doc
sudo cpanm --quiet --notest App::Sqitch
sqitch --version
```

### Setup git project

```bash
mkdir sead_db_change_control
cd sead_db_change_control
git init .
touch README.md
git add .
git commit -am "SEAD db change control project inititalized"
```

### Setup Sqitch project

```bash
sqitch init sead_db_change_control --uri https://github.com/humlab-sead/sead_db_change_control --engine pg
sqitch config --user engine.pg.client /usr/bin/psql
```


In [ ]:
mkdir clearinghouse
sqitch init clearinghouse --uri https://github.com/humlab-sead/clearinghouse --engine pg --top_dir ./clearinghouse --plan-file clearinghouse.plan


In [4]:
%%bash
#sqitch init report --uri https://github.com/humlab-sead/report --engine pg -C ./report
mkdir sead_api
sqitch init sead_api --uri https://github.com/humlab-sead/sead-api --engine pg -C sead_api

Created sqitch.conf
Created sqitch.plan
Created deploy/
Created revert/
Created verify/


### Setup global defaults for user
These settings are global user settings specific for each computing environment.

In [ ]:
%%bash
sqitch config --user engine.pg.client /usr/bin/psql
sqitch config --user user.name 'Roger Mähler'
sqitch config --user user.email 'roger.mahler@umu.se'

### Setup default user templates
These settings are global user settings specific for each computing environment.

In [ ]:
%%bash

mkdir -p ~/.sqitch/templates/deploy
mkdir -p ~/.sqitch/templates/revert 
mkdir -p ~/.sqitch/templates/verify

tmpldir=`sqitch --etc-path`/templates

cp $tmpldir/deploy/pg.tmpl ~/.sqitch/templates/deploy/pg.tmpl
cp $tmpldir/revert/pg.tmpl ~/.sqitch/templates/revert/pg.tmpl
cp $tmpldir/verify/pg.tmpl ~/.sqitch/templates/verify/pg.tmpl

chmod -R +w ~/.sqitch/templates


### Changfe default 'pg' template
These settings are global user settings specific for each computing environment.

In [ ]:
%%file  ~/.sqitch/templates/deploy/pg.tmpl
-- Deploy [% project %]:[% change %] to [% engine %]
[% FOREACH item IN requires -%]
-- requires: [% item %]
[% END -%]
[% FOREACH item IN conflicts -%]
-- conflicts: [% item %]
[% END -%]

/****************************************************************************************************************
  Author        Roger Mähler
  Date          2019-01-01
  Description   
  Prerequisites 
  Reviewer      
  Approver      
  Idempotent    Yes
  Notes
*****************************************************************************************************************/

begin;
do $$
begin

    begin
    
        if sead_utility.column_exists('public'::text, 'table_name'::text, 'column_name'::text) = TRUE then
            raise exception SQLSTATE 'GUARD';
        end if;
        
        -- insert your DDL code here
        
    exception when sqlstate 'GUARD' then
        raise notice 'ALREADY EXECUTED';
    end;
    
end $$;
commit;


#### Install PostgreSQL Kernel
```bash
sudo pip3 install postgres_kernel
```

In [1]:
%%bash
pg_dump --help

pg_dump dumps a database as a text file or to other formats.

Usage:
  pg_dump [OPTION]... [DBNAME]

General options:
  -f, --file=FILENAME          output file or directory name
  -F, --format=c|d|t|p         output file format (custom, directory, tar,
                               plain text (default))
  -j, --jobs=NUM               use this many parallel jobs to dump
  -v, --verbose                verbose mode
  -V, --version                output version information, then exit
  -Z, --compress=0-9           compression level for compressed formats
  --lock-wait-timeout=TIMEOUT  fail after waiting TIMEOUT for a table lock
  -?, --help                   show this help, then exit

Options controlling the output content:
  -a, --data-only              dump only the data, not the schema
  -b, --blobs                  include large objects in dump
  -c, --clean                  clean (drop) database objects before recreating
  -C, --create                 include commands to create data

In [3]:
%%bash
#SEAD_SERVER=`cat ~/.default.sead.server`
#SEAD_USERNAME=`cat ~/.default.sead.username`
pg_dump --file "humlab_utility.sql" --host `cat ~/.default.sead.server` --username `cat  ~/.default.sead.username` --no-password \
    --verbose --format=p --no-owner --no-privileges --no-tablespaces \
    --no-unlogged-table-data --column-inserts --schema "humlab_utility" "sead_bugs_import_20170814"

pg_dump: last built-in OID is 16383
pg_dump: reading extensions
pg_dump: identifying extension members
pg_dump: reading schemas
pg_dump: reading user-defined tables
pg_dump: reading user-defined functions
pg_dump: reading user-defined types
pg_dump: reading procedural languages
pg_dump: reading user-defined aggregate functions
pg_dump: reading user-defined operators
pg_dump: reading user-defined access methods
pg_dump: reading user-defined operator classes
pg_dump: reading user-defined operator families
pg_dump: reading user-defined text search parsers
pg_dump: reading user-defined text search templates
pg_dump: reading user-defined text search dictionaries
pg_dump: reading user-defined text search configurations
pg_dump: reading user-defined foreign-data wrappers
pg_dump: reading user-defined foreign servers
pg_dump: reading default privileges
pg_dump: reading user-defined collations
pg_dump: reading user-defined conversions
pg_dump: reading type casts
pg_dump: reading transforms
pg_d

In [ ]:
import sqlalchemy
with open('~/.default.sead.server') as f:
    server_name = f.read()
with open('~/.default.sead.username') as f:
    user_name = f.read()
postgres_engine = sqlalchemy.create_engine("postgresql://{}:@{}/postgres".format(user_name, server_name))
%load_ext sql_magic
%config SQL.conn_name = 'postgres_engine'
%config SQL.output_result = False
%config SQL.notify_result = True

In [ ]:
%%read_sql df_tables
select *
from information_schema.databases
--where table_schema = 'public'
--  and table_type = 'BASE TABLE'

In [ ]:
df_tables

In [ ]:
%%file rename_change.bash
#/bin/bash

function usage() {
    echo "usage: $0 change-name new-change-name"
    exit 0 
}

if [ $# != 2 ]; then
    usage
fi
    
if [ ! -f ./deploy/$1.sql ]; then
    echo "error: no such change file exists"
    exit 0 
fi

if [ -f ./deploy/$2.sql ]; then
    echo "error: file exists, chose another name"
    exit 0 
fi
    
mv --force --no-clobber ./deploy/$1.sql ./deploy/$2.sql
mv --force --no-clobber ./revert/$1.sql ./revert/$2.sql
mv --force --no-clobber ./verify/$1.sql ./verify/$2.sql

sed -i 's/$1/$2/g' sqitch.plan


In [ ]:
%%bash
./rename_change.bash CSR_20190406_REFACTOR_BIBLIOGRAPHIC_MODEL CSR_20170101_REFACTOR_BIBLIOGRAPHIC_MODEL

#### Change tag format to "CS_GROUP_YYYYMMDD_DESCRIPTION"

In [ ]:
import glob
import os

paths = glob.glob('./deploy/*.sql')
files = [ os.path.split(path)[1] for path in paths ]
tags = [ os.path.splitext(file)[0] for file in files ]

#for folder in [ 'deploy', 'revert', 'verify' ]:
#    for tag in tags:
#        path = os.path.join('.', folder, tag + '.sql')

# [ (tag, tag) for tag in tags ]


In [ ]:
import os
import shutil

tag_renames = [
    ('CS_SITE_20170911_ANALYSIS_ENTITY_ALTER_AGES_PRECISION', 'CS_ANALYSIS_ENTITY_20170911_ALTER_AGES_PRECISION')
]


def replace_tags_in_plan(project, tags, plan_file="sqitch.plan"):
    
    plan_path = os.path.join('.', project, plan_file)
    
    with open(plan_path, "rt") as f:
        plan = f.read()
    
    for tag_old, tag_new in tags:
        
        plan = plan.replace(tag_old, tag_new)
        
        for folder in [ 'deploy', 'revert', 'verify' ]:
            path_old = os.path.join('.', project, folder, tag_old + '.sql')
            path_new = os.path.join('.', project, folder, tag_new + '.sql')
            try:
                shutil.move(path_old, path_new)
            except:
                pass
        
    with open(plan_path, "wt") as f:
        f.write(plan)
        
    
replace_tags_in_plan('general', tag_renames)